In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/sample (1).csv
/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/test.csv (1)/test.csv
/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/movies.csv (1)/movies.csv
/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/train.csv (1)/train.csv


# **Imports**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#SKLEARN IMPORTS
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import hstack

# **1. Loading the data**

# *movies.csv*

In [4]:
#LOADING THE DATA
movie_data_path = '/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/movies.csv (1)/movies.csv'
movie_data = pd.read_csv(movie_data_path)
movie_data.shape

(143258, 14)

In [5]:
#EXAMINING THE DATA
movie_data.head()

,movieid,title,audienceScore,rating,ratingContents,releaseDateTheaters,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,boxOffice,distributor,soundType
0,han_solo_heroic_myth_wonder_woman,Han Solo Heroic Myth Wonder Woman,50.0,NaN,NaN,NaN,2018-08-25,75.0,"Comedy, Horror, Sci-fi",English,Claude Nicolet,NaN,NaN,NaN
1,voyage_tyler_durden_han_solo_stardust,Voyage Tyler Durden Han Solo Stardust,NaN,NaN,NaN,NaN,2020-02-11,114.0,Drama,English,Nathan Haack,NaN,NaN,NaN
2,norman_bates_ferris_bueller_hermione_granger_v...,Norman Bates Ferris Bueller Hermione Granger V...,43.0,NaN,NaN,NaN,NaN,120.0,Drama,Korean,Theresa Smith,NaN,NaN,NaN
3,elegant_hermione_granger,Elegant Hermione Granger,60.0,NaN,NaN,NaN,2020-10-23,90.0,"Action, Mystery & thriller",English,Donald Lewis,NaN,NaN,NaN
4,adventure_rocky_balboa,Adventure Rocky Balboa,70.0,NaN,NaN,NaN,2017-03-27,80.0,"Fantasy, Adventure, Animation",English,Howard Simms,NaN,NaN,NaN


# *train.csv*

In [6]:
#LOADING & CHECKING SHAPE OF train.csv
train_data_path='/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/train.csv (1)/train.csv'
train_data=pd.read_csv(train_data_path)
train_data.shape

(162758, 5)

In [7]:
train_data.head()

,movieid,reviewerName,isFrequentReviewer,reviewText,sentiment
0,marvelous_pirate,Benjamin Henry,False,Henry Selick’s first movie since 2009’s Corali...,POSITIVE
1,tony_montana_frodo_baggins_v_rocky_balboa,Felicia Lopez,False,With a cast that reads like the Vogue Oscar pa...,NEGATIVE
2,darth_vader_katniss_everdeen_sorcerer_donnie_d...,Mr. Charles Burgess,True,Creed II does not give us anything but another...,POSITIVE
3,lara_croft_glimmer,Ryan Barrett,False,"I know what you're thinking, but this is no Li...",POSITIVE
4,jason_bourne_surreal_the_terminator_indiana_jones,Alexander Glover,False,Director Fernando Meirelles tells the story wi...,POSITIVE


# *JOINING movies.csv AND train.csv*

In [8]:
#JOINING train.csv AND movies.csv
joined_data = pd.merge(movie_data,train_data)
joined_data.shape

(525277, 18)

In [9]:
#JOINED DATA
joined_data.head()

,movieid,title,audienceScore,rating,ratingContents,releaseDateTheaters,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,boxOffice,distributor,soundType,reviewerName,isFrequentReviewer,reviewText,sentiment
0,james_bond_courageous,James Bond Courageous,65.0,PG-13,"['Injury Images', 'Brief Drug Use', 'Thematic ...",2018-06-01,2018-08-21,120.0,"Adventure, Drama, Romance",English,Leroy Fernandes,$31.4M,STX Films,NaN,Karen Hamilton,False,This is nowhere near the level of other great ...,POSITIVE
1,james_bond_courageous,James Bond Courageous,65.0,PG-13,"['Injury Images', 'Brief Drug Use', 'Thematic ...",2018-06-01,2018-08-21,120.0,"Adventure, Drama, Romance",English,Leroy Fernandes,$31.4M,STX Films,NaN,Jessica Best,False,If you're feeling adventurous and looking for ...,POSITIVE
2,james_bond_courageous,James Bond Courageous,65.0,PG-13,"['Injury Images', 'Brief Drug Use', 'Thematic ...",2018-06-01,2018-08-21,120.0,"Adventure, Drama, Romance",English,Leroy Fernandes,$31.4M,STX Films,NaN,Rebekah Gomez,False,"Mercifully, Vincent Cassel provides a strong l...",POSITIVE
3,james_bond_courageous,James Bond Courageous,65.0,PG-13,"['Injury Images', 'Brief Drug Use', 'Thematic ...",2018-06-01,2018-08-21,120.0,"Adventure, Drama, Romance",English,Leroy Fernandes,$31.4M,STX Films,NaN,Paul Sullivan,False,Like the storm that destroys the best laid pla...,POSITIVE
4,james_bond_courageous,James Bond Courageous,65.0,PG-13,"['Injury Images', 'Brief Drug Use', 'Thematic ...",2018-06-01,2018-08-21,120.0,"Adventure, Drama, Romance",English,Leroy Fernandes,$31.4M,STX Films,NaN,Stacey Malone,True,Less a proper coming-of-age movie than a posh ...,NEGATIVE


***PRINTING FEATURES AND LABEL***

In [10]:
#FEATURES AND LABELS 
features_list = joined_data.columns[:-1].values
label = joined_data.columns[-1]

print("Features: " , features_list)
print("Label: " , label)

Features:  ['movieid' 'title' 'audienceScore' 'rating' 'ratingContents'
 'releaseDateTheaters' 'releaseDateStreaming' 'runtimeMinutes' 'genre'
 'originalLanguage' 'director' 'boxOffice' 'distributor' 'soundType'
 'reviewerName' 'isFrequentReviewer' 'reviewText']
Label:  sentiment


In [11]:
#NUMBER OF POSITIVE AND NEGATIVE REVIEWS
joined_data['sentiment'].value_counts()

sentiment
POSITIVE    350583
NEGATIVE    174694
Name: count, dtype: int64

In [12]:
# joined_data.describe()

In [13]:
#UNIQUE VALUES
joined_data.nunique()

movieid                  16812
title                    16812
audienceScore              100
rating                       7
ratingContents            5371
releaseDateTheaters       5066
releaseDateStreaming      3249
runtimeMinutes             238
genre                     1339
originalLanguage            96
director                 13038
boxOffice                 3918
distributor               1913
soundType                  487
reviewerName              4482
isFrequentReviewer           2
reviewText              155071
sentiment                    2
dtype: int64

In [14]:
#MOVIE LANGUAGE
joined_data['originalLanguage'].unique()

array(['English', 'Portuguese (Brazil)', 'Polish', 'Spanish',
       'French (Canada)', 'German', nan, 'Dutch', 'Hindi',
       'French (France)', 'Swedish', 'Korean', 'Hebrew', 'Finnish',
       'Chinese', 'Persian', 'Italian', 'Armenian', 'English (Australia)',
       'English (United Kingdom)', 'Malayalam', 'Turkish', 'Russian',
       'Tamil', 'Ukrainian', 'Bambara', 'Catalan', 'Norwegian', 'French',
       'Japanese', 'Arabic', 'Unknown language', 'Spanish (Spain)',
       'Urdu', 'Bulgarian', 'Icelandic', 'Amharic', 'Danish', 'Telugu',
       'Kannada', 'Hungarian', 'Filipino', 'Estonian', 'Belarusian',
       'Tagalog', 'Greek', 'Portuguese', 'Malay', 'Indonesian', 'Thai',
       'Inuktitut', 'Marathi', 'Croatian', 'Yiddish', 'Punjabi',
       'Romanian', 'Czech', 'Slovak', 'Slovenian', 'Afrikaans',
       'Dzongkha', 'Bangla', 'Gujarati', 'Bosnian', 'Albanian',
       'Vietnamese', 'Georgian', 'Serbian', 'Scottish Gaelic',
       'Lithuanian', 'Zulu', 'Lao', 'Pashto', 'Mongolia

# **2.Cleaning & Preprocessing the data**

In [15]:
#SEARCHING FOR NULL VALUES
joined_data.isnull().sum()

movieid                      0
title                        0
audienceScore           300829
rating                  390791
ratingContents          390791
releaseDateTheaters     342951
releaseDateStreaming    275844
runtimeMinutes          222688
genre                   217333
originalLanguage        220455
director                     0
boxOffice               380216
distributor             355592
soundType               415919
reviewerName                 0
isFrequentReviewer           0
reviewText               23817
sentiment                    0
dtype: int64

In [16]:
# DROPPING THE 'boxOffice' COLUMN
data = joined_data.drop(['boxOffice'], axis=1)

In [17]:
# EXTRACTING FEATURES (X) AND LABELS (y)
X = data.drop(['sentiment'], axis=1)  
y = data['sentiment']  

In [18]:
#ENCDING POSTIVE AND NEGATIVE VALUES AS NUMERICAL VALUES
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [19]:
#CHECKING SHAPES OF X AND Y
X.shape, y_encoded.shape

((525277, 16), (525277,))

In [20]:
#IMPUTING NUMERICAL VALUES
num_imputer = SimpleImputer(strategy='median')

numerical_cols = ['audienceScore', 'runtimeMinutes']  
imputed_num_data = num_imputer.fit_transform(data[numerical_cols])

In [21]:
#IMPUTING CATEGORICAL VALUES
cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')

categorical_cols = ['rating', 'ratingContents', 'releaseDateTheaters', 'releaseDateStreaming', 'genre', 'originalLanguage', 'distributor', 'soundType']  
imputed_cat_data = cat_imputer.fit_transform(data[categorical_cols])

In [22]:
ordinal_encoder = OrdinalEncoder()

oe_cat_data = ordinal_encoder.fit_transform(data[categorical_cols])

In [23]:
#SPLITTING THE DATA IN TRAIN AND TEST SETS

X_train, X_test, y_train, y_test = train_test_split(X,y_encoded, test_size = 0.2, random_state = 48)

In [24]:
#FIRST WE IMPUTE MISSING VALUES WITH SPACE

X_train['reviewText'].fillna('', inplace=True)
X_test['reviewText'].fillna('', inplace=True)

In [25]:
#Tfidf VECTORIZER FOR REVIEW TEXT

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train['reviewText'])
X_test_tfidf = tfidf_vectorizer.transform(X_test['reviewText'])

In [26]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train_tfidf)

In [27]:
X_train_tfidf.shape, y_train.shape

((420221, 61417), (420221,))

In [28]:
X_test.shape

(105056, 16)

# **3. Data Visualisation**

In [29]:
# sns.pairplot(data)

In [30]:
# sns.relplot(x = 'audienceScore', y = 'runtimeMinutes', hue = 'originalLanguage', data = joined_data)

In [31]:
# sns.relplot(x = 'audienceScore', y = 'runtimeMinutes', hue = 'sentiment', data = joined_data)

# **3. Training the model**

# *MODEL 1* (*Dummy Classifier*)

In [32]:
dummy_clf = DummyClassifier(strategy = 'most_frequent')

In [33]:
#FITTING THE MODEL
dummy_clf.fit(X_train_tfidf, y_train)

DummyClassifier(strategy='most_frequent')

In [34]:
#PREDICTION ON train_test_split
y_pred_dummy_clf = dummy_clf.predict(X_test_tfidf)

f1Score_dummy = f1_score(y_test, y_pred_dummy_clf)

print("F1 Score on dummy classifier : ", f1Score_dummy)

F1 Score on dummy classifier :  0.8012847941305675


# *MODEL 2 (Logistic Regression)*

In [35]:
# #LOGISTIC REGRESSION
# log_reg = LogisticRegression(max_iter = 1000)
# log_reg.fit(X_train_tfidf, y_train)

In [36]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=10000),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]})

In [37]:
best_params = grid_search.best_params_
print(best_params)
best_log_reg = grid_search.best_estimator_
print(best_log_reg)

{'C': 10}
LogisticRegression(C=10, max_iter=10000)


In [38]:
y_pred = best_log_reg.predict(X_test_tfidf)
f1_gs_log = f1_score(y_test, y_pred)
print("Logistic Regression with GS f1 score:", f1_gs_log)

Logistic Regression with GS f1 score: 0.934963080388803


In [39]:
# # PREDICTION ON joined_data AND f1 SCORE
# y_pred_log_reg = log_reg.predict(X_test_tfidf)
# f1_log_reg = f1_score(y_test, y_pred_log_reg)
# print("Logistic Regression f1 score:", f1_log_reg)

# *MODEL 3 (Random Forest Classifier)*

In [40]:
# RANDOMFOREST CLASSIFIER
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)  
rf_clf.fit(X_train_tfidf, y_train)

RandomForestClassifier(random_state=42)

In [41]:
# PREDICTION AND f1 SCORE
y_pred_rf = rf_clf.predict(X_test_tfidf)
f1_rf = f1_score(y_test, y_pred_rf)
print("Random Forest f1 Score:",f1_rf)

Random Forest f1 Score: 0.9435817665359124


# *MODEL 4 (SVC)*

In [42]:
# svm_model = SVC(kernel='linear', C=1.0)  
# svm_model.fit(X_train_tfidf, y_train)

In [43]:
# y_pred_svm = svm_model.predict(X_test_tfidf)
# f1_svm = f1_score(y_test, y_pred_svm)
# print("SVM f1 Score:", f1_svm)

# **4. Predicting test results**

# *LOADING test.csv*

In [44]:
#LOADING test.csv file
test_data_path='/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/test.csv (1)/test.csv'
test_data=pd.read_csv(test_data_path) 

#SHAPE OF test.csv
test_data.shape

(55315, 4)

In [45]:
#SEARCHING FOR NULL VALUES
test_data.isnull().sum()

movieid            0
reviewerName       0
isTopCritic        0
reviewText      2510
dtype: int64

In [46]:
#IMPUTING NULL VALUES IN test.csv AS BLANK SPACE
test_data['reviewText'].fillna('', inplace=True)

# *PREDICTION ON test.csv*

***PREDICTION ON DUMMY CLASSIFIER (Score = 0.66846)***

In [47]:
# #PREDICTION on test.csv
# X_test_tfidf = tfidf_vectorizer.transform(test_data['reviewText'])
# y_test_pred = dummy_clf.predict(X_test_tfidf)
# # test_data['predicted_sentiment'] = y_test_pred.map({1: 'positive', 0: 'negative'})

# predicted_labels = np.where(y_test_pred == 1, 'POSITIVE', 'NEGATIVE')

# test_data['predicted_sentiment'] = predicted_labels

***PREDICTION USING LOGISTIC REGRESSION (Score = 0.79211)***

In [48]:
# #PREDICTION on test.csv
# X_test_tfidf = tfidf_vectorizer.transform(test_data['reviewText'])
# y_test_pred = log_reg.predict(X_test_tfidf)
# # test_data['predicted_sentiment'] = y_test_pred.map({1: 'positive', 0: 'negative'})

# predicted_labels = np.where(y_test_pred == 1, 'POSITIVE', 'NEGATIVE')

# test_data['predicted_sentiment'] = predicted_labels

In [49]:
#PREDICTION on test.csv
X_test_tfidf = tfidf_vectorizer.transform(test_data['reviewText'])
y_test_pred = best_log_reg.predict(X_test_tfidf)
# test_data['predicted_sentiment'] = y_test_pred.map({1: 'positive', 0: 'negative'})

predicted_labels = np.where(y_test_pred == 1, 'POSITIVE', 'NEGATIVE')

test_data['predicted_sentiment'] = predicted_labels

***PREDICTION USING RANDOM FOREST CLASSIFIER (Score = 0.73052)***

In [50]:
# #PREDICTION on test.csv
# X_test_tfidf = tfidf_vectorizer.transform(test_data['reviewText'])
# y_test_pred = rf_clf.predict(X_test_tfidf)
# # test_data['predicted_sentiment'] = y_test_pred.map({1: 'positive', 0: 'negative'})

# predicted_labels = np.where(y_test_pred == 1, 'POSITIVE', 'NEGATIVE')

# test_data['predicted_sentiment'] = predicted_labels

# **SUBMISSION**

In [51]:
submission_df = pd.DataFrame({
    'id': range(len(test_data)), 
    'sentiment': test_data['predicted_sentiment']  
})

submission_df.to_csv('submission.csv', index=False)